<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [209]:
import pandas as pd
import psycopg2
import warnings

In [210]:
warnings.filterwarnings("ignore")

In [212]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [213]:
query_3_1 = f'''select count(*)
                from public.vacancies
'''

In [214]:
df = pd.read_sql_query(query_3_1, connection)
df

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [215]:
query_3_2 = f'''select count(*)
                from public.employers
'''

In [216]:
df = pd.read_sql_query(query_3_2, connection)
df

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [217]:
query_3_3 = f'''select count(*)
                from public.areas
'''

In [218]:
df = pd.read_sql_query(query_3_3, connection)
df

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [219]:
query_3_4 = f'''select count(*)
                from public.industries
'''

In [220]:
df = pd.read_sql_query(query_3_4, connection)
df

,count
0,294


***

Как видим, количество вакансий примерно в 2 раза превышает количество работодателей, что вполне ожидаемо, ведь, как правило, одной и той же организации для успешного функционирования требуются специалисты из разных областей. В базе представлено 1362 региона - это говорит о том, что мы охватываем довольно широкую географию. Также у нас имеются 294 различные сферы деятельности (т.е. в среднем по 167 вакансий на каждую сферу, но нужно, конечно, проводить дополнительный анализ). Это тоже довольно неплохая статистика, у соискателей есть возможность выбора и больше шансов куда-то устроиться в принципе (хотя, опять же, зависит от количества соискателей).

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [221]:
query_4_1 = f'''select 
                    a.name as area,
                    count(v.id) as vacancies_count
                from 
                    public.areas as a
                    join public.vacancies as v on v.area_id = a.id
                group by a.name
                order by count(v.id) desc
                limit 10
'''

In [222]:
df = pd.read_sql_query(query_4_1, connection)
df

,area,vacancies_count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [223]:
query_4_2 = f'''select 
                    count(*)
                from 
                    public.vacancies
                where salary_from is not null
                    or salary_to is not null
'''

In [224]:
df = pd.read_sql_query(query_4_2, connection)
df

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [225]:
query_4_3 = f'''select 
                    avg(salary_from) as mean_salary_from,
                    avg(salary_to) as mean_salary_to
                from 
                    public.vacancies
'''

In [226]:
df = pd.read_sql_query(query_4_3, connection)
print(round(df['mean_salary_from']))
print(round(df['mean_salary_to']))

0    71065.0
Name: mean_salary_from, dtype: float64
0    110537.0
Name: mean_salary_to, dtype: float64


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [227]:
query_4_4 = f'''select 
                    schedule,
                    employment,
                    count(*) as vacancies_count
                from 
                    public.vacancies
                group by schedule, employment
                order by count(*) desc
'''

In [228]:
df = pd.read_sql_query(query_4_4, connection)
df

,schedule,employment,vacancies_count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [229]:
query_4_5 = f'''select 
                    experience,
                    count(*) as vacancies_count
                from 
                    public.vacancies
                group by experience
                order by count(*) 
'''

In [230]:
df = pd.read_sql_query(query_4_5, connection)
df

,experience,vacancies_count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


4* Выясним, в каких отраслях можно найти больше всего вакансий

In [231]:
ad_query = f'''select 
                    i.name as industry,
                    count(v.id) as vacancies_count
                from 
                    public.employers_industries as ei
                    join public.industries as i on i.id = ei.industry_id
                    join public.vacancies as v on v.employer_id = ei.employer_id
                group by i.id
                order by count(v.id) desc
                limit 5
                
'''

In [232]:
df = pd.read_sql_query(ad_query, connection)
df

,industry,vacancies_count
0,Разработка программного обеспечения,12499
1,"Системная интеграция, автоматизации технологи...",11034
2,"Интернет-компания (поисковики, платежные систе...",6413
3,Банк,2742
4,Мобильная связь,1585


***

Видим, что основная доля вакансий сосредоточена в крупных городах и столицах стран СНГ: в топ-5 по числу вакансий входят Москва, Санкт-Петербург, Минск, Новосибирск и Алматы (на данные города приходится примерно 30% всех вакансий). И это неудивительно, ведь именно в крупных городах расположено большинство предпрниятий и бизнес-центров. Примерно 72% вакансий имеют тип рабочего графика "полный день" и тип трудоустройства "полная занятость". Это тоже вполне ожидаемо, ведь работодателями выгоднее нанимать сотрудников, которые полностью будут погружены в дела компании и всегда будут присутствовать на своем рабочем месте. Наиболее частые пожелания работодателей по опыту работы (53% вакансий) - от 1 до 3 лет. Это вполне достаточно, чтобы погрузиться в предметную область и как следует в ней разобраться. Для ряда должностей, связанных с занимаением каких-либо руководящих позиций, может потребоваться уже больше опыта - от 3 до 6 лет (около 29% вакансий). Требование опыта более 6 лет кажется уже избыточным. Также можно заметить, что нижняя граница зарплатной вилки имеет довольно высокое среднее значение - более 70 тыс. рублей в месяц. Вероятно, это также связано с тем, что основная доля вакансий сосредоточена в крупных городах и столицах (например, по данным на начало 2022 года средняя зарплата в Москве составляла 116 354 рублей в месяц). Основное количество вакансий приходится на IT-технологии (разработка ПО, системная интеграция и автоматизация, Интернет-компания, мобильная связь) и банковскую сферу. На эти топ-5 отраслей приходится примерно 70% от общего числа вакансий. Это тоже вполне естественно, ведь компьютерные технологии сейчас в том или ином виде присутствуют везде, да и финансовый сектор играет немаловажную роль в жизни каждого из нас: с банковскими операциями приходится сталкиваться практически всем.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [233]:
query_5_1 = f'''select 
                    e.name, 
                    count(v.id) as vacancies_count
                from 
                    public.employers as e
                    join public.vacancies as v on v.employer_id = e.id
                group by e.name
                order by count(v.id) desc
                limit 5
'''

In [234]:
df = pd.read_sql_query(query_5_1, connection)
df

,name,vacancies_count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [235]:
query_5_2 = f'''select 
                    a.name,
                    count(e.name) as employers_count
                from 
                    public.areas as a
                    left join public.vacancies as v on v.area_id = a.id
                    join public.employers as e on e.area = a.id
                group by a.name
                having count(v.id) = 0
                order by count(e.name) desc
                limit 10
                
'''

In [236]:
df = pd.read_sql_query(query_5_2, connection)
df

,name,employers_count
0,Россия,410
1,Казахстан,207
2,Московская область,75
3,Краснодарский край,19
4,Беларусь,18
5,Ростовская область,18
6,Азербайджан,17
7,Республика Татарстан,16
8,Нижегородская область,16
9,Узбекистан,15


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [237]:
query_5_3 = f'''select 
                    e.name as employer,
                    count(distinct v.area_id) as vacancies_areas_count
                from 
                    public.employers as e
                    join public.vacancies as v on v.employer_id = e.id
                group by e.id
                order by count(v.area_id) desc
                limit 10            
                
'''

In [238]:
df = pd.read_sql_query(query_5_3, connection)
df

,employer,vacancies_areas_count
0,Яндекс,181
1,Ростелеком,152
2,Тинькофф,43
3,СБЕР,24
4,Газпром нефть,24
5,ИК СИБИНТЕК,46
6,МТС,55
7,DataArt,2
8,Совкомбанк Технологии,26
9,Первый Бит,43


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [239]:
query_5_4 = f'''select 
                    count(*)
                from
                    (select 
                        id
                    from public.employers 
                
                    except
                
                    select
                        employer_id
                    from public.employers_industries) as help_table
'''
#или можно было бы сделать один запрос, который обернут в help_table, а потом взять df.shape[0]

In [240]:
df = pd.read_sql_query(query_5_4, connection)
df

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [241]:
query_5_5 = f'''select 
                    e.name,
                    count(ei.industry_id)
                from 
                    public.employers as e
                    join public.employers_industries as ei on ei.employer_id = e.id   
                group by e.id
                having count(ei.industry_id) = 4
                order by e.name
                limit 5
                
'''

In [242]:
df = pd.read_sql_query(query_5_5, connection)
df

,name,count
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [243]:
query_5_6 = f'''select 
                   count(ei.employer_id)
                from 
                    public.industries as i
                    join public.employers_industries as ei on ei.industry_id = i.id 
                group by i.name
                having i.name = 'Разработка программного обеспечения'
                
'''

In [244]:
df = pd.read_sql_query(query_5_6, connection)
df

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [245]:
millionaire_cities = ( 
                      'Москва',
                      'Санкт-Петербург',
                      'Новосибирск',
                      'Екатеринбург',
                      'Казань',
                      'Нижний Новгород',
                      'Челябинск',
                      'Красноярск',
                      'Самара',
                      'Уфа',
                      'Ростов-на-Дону',
                      'Омск',
                      'Краснодар',
                      'Воронеж',
                      'Пермь',
                      'Волгоград'
                     )


In [246]:
query_5_7 = f'''select 
                   a.name as region,
                   count(v.id) as vacancies_count
                from 
                    public.employers as e
                    join public.vacancies as v on v.employer_id = e.id
                    join public.areas as a on a.id = v.area_id 
                where e.name = 'Яндекс' 
                    and a.name in {millionaire_cities}
                group by a.name
                
                union all
                
                select 
                   'Total',
                    count(*)
                from 
                    public.employers as e
                    join public.vacancies as v on v.employer_id = e.id
                    join public.areas as a on a.id = v.area_id 
                where e.name = 'Яндекс' 
                    and a.name in {millionaire_cities}
                order by vacancies_count
                
'''

In [247]:
df = pd.read_sql_query(query_5_7, connection)
df

,region,vacancies_count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

Видим, что работодатели преимущественно размещают свои вакансии в городах-миллионерах и столицах, что неудивительно, ведь именно там обычно располагается большинство крупных компаний, административный учреждений и бизнес-центров. Также эти города являются центром притяжения для населения близлежащих регионов (а то и всей страны в целом), в них проживает больше людей и для их жизнеобеспечения требуется больше ресуросов. Среди работодателей топ-5 лидеров по количеству размещенных вакансий занимают такие компании, как "Яндекс", "Ростелеком", "Сбер", "Тинькофф и Газпром, что тоже вполне ожидаемо, ведь это крупные корпорации, имеющие филиалы в нескольких городах и специализирующиеся на нескольких смежных областях. Можем заметить, что данные были собраны не очень аккуратно. Например, формат указание региона регистрации компании явно не был согласован, и кто-то из опрошеных указал только страну, кто-то - название конкретного региона, кто-то - конкретный город. Это вносит дополнительные сложности в процесс анализа данных и приводит к некорректным результатам. Например, мы получили, что у нас в Ростовской области нет вакансий, в то время как в одном Ростове-на-Дону есть 25 вакансий компании "Яндекс". То, что у 36% работодателей не указана сфера деятельности, тоже наводит на мысль о качестве собранных данных.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [248]:
query_6_1 = f'''select 
                   count(*)
                from 
                    public.vacancies 
                where lower(name) like '%data%'
                    or lower(name) like '%данн%'
                
'''

In [249]:
df = pd.read_sql_query(query_6_1, connection)
df

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [250]:
help_table_query = f'''select 
                           *
                        into dst_vacancies
                        from 
                            public.vacancies 
                        where lower(name) like '%data scientist%'
                            or lower(name) like '%data science%'
                            or lower(name) like '%исследователь данных%'
                            or (name like '%ML%' and name not like '%HTML%')
                            or lower(name) like '%machine learning%'
                            or lower(name) like '%машинн%обучен%'
                
'''

In [252]:
query_6_2 = f'''select 
                   count(*)
                from 
                    dst_vacancies
                where lower(name) like '%junior%'
                    or experience = 'Нет опыта'
                    or employment = 'Стажировка'
'''

In [253]:
df = pd.read_sql_query(query_6_2, connection)
df

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [254]:
query_6_3 = f'''select 
                   count(*)
                from 
                    dst_vacancies
                where lower(key_skills) like '%sql%'
                    or lower(key_skills) like '%postgres%'
                
'''

In [255]:
df = pd.read_sql_query(query_6_3, connection)
df

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [256]:
query_6_4 = f'''select 
                   count(*)
                from 
                    dst_vacancies
                where lower(key_skills) like '%python%'
                
'''

In [257]:
df = pd.read_sql_query(query_6_4, connection)
df

,count
0,351


In [258]:
query_6_4_ad = f'''select 
                   count(*)
                from 
                    dst_vacancies
                where lower(key_skills) like '%python%'
                    or lower(key_skills) like '%flask%'
                    or lower(key_skills) like '%scikit-learn%'
                    or lower(key_skills) like '%pandas%'
                    or lower(key_skills) like '%numpy%'
                
'''

In [259]:
df = pd.read_sql_query(query_6_4_ad, connection)
df

,count
0,353


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [260]:
query_6_5 = f'''select 
                   round(avg(length(key_skills) - length(replace(key_skills, chr(9), '')) + 1), 2) as average_skills
                from 
                    dst_vacancies
                
                
'''

In [261]:
df = pd.read_sql_query(query_6_5, connection)
df

,average_skills
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [267]:
query_6_6 = f'''select 
                   experience,
                   round(avg(case when salary_from is not null and salary_to is not null 
                            then (salary_from + salary_to) / 2
                        when salary_from is not null
                            then salary_from
                        else salary_to
                   end))
                from 
                    dst_vacancies
                where salary_from is not null
                    or salary_to is not null
                group by experience
                    
                limit 20
                
                
'''

In [268]:
df = pd.read_sql_query(query_6_6, connection)
df

,experience,round
0,От 3 до 6 лет,243115.0
1,От 1 года до 3 лет,139675.0
2,Нет опыта,74643.0


6* Выясним, как часто от специалистов по Data Science работодатели требуют знания математики

In [269]:
ad_query = f'''select 
                   count(*)
                from 
                    dst_vacancies
                where lower(key_skills) like '%математ%'
                    or lower(key_skills) like '%алгебра%'
                    or lower(key_skills) like '%теория вероятностей%'
                    or lower(key_skills) like '%статистика%'
                    or lower(key_skills) like '%дифференциал%'
                    or lower(key_skills) like '%численные методы%'
                
'''

In [270]:
df = pd.read_sql_query(ad_query, connection)
df

,count
0,112


6** Выясним, какие компании чаще всего ищут специалистов по Data Science

In [271]:
ad_query = f'''select 
                   e.name,
                   count(v.id) as ds_vacancies_count
                from 
                    dst_vacancies as v
                    join public.employers as e on e.id = v.employer_id
                group by e.id
                order by count(v.id) desc
                limit 10
'''

In [272]:
df = pd.read_sql_query(ad_query, connection)
df

,name,ds_vacancies_count
0,СБЕР,37
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11
5,Яндекс,9
6,EvenBet Gaming,9
7,МегаФон,8
8,Andersen,7
9,Ozon,6


***

Видим, что специалисты в области Data Science могут претендовать на очень высокую зарплату: в районе 75 тыс.руб. в месяц для новичков, в районе 140 тыс. рублей для специалистов со стажем и в около 243 тыс. руб. для опытных специалистов. И это весьма ожидаемо, ведь общеизвестно, что  data scinetist является одной из наиболее высокооплачиваемых профессий в наши дни. Одним из ключевых навыков для промышленного специалиста в области DS и ML является владение языком python. В явном виде этот навык указан в 20% вакансий, но, я думаю, реальное количество будет больше, т.к. зачастую работодатель в требованиях указывает владения конкретными питоновскими фреймворками и библиотеками(scikit-learn, pandas, nltk и т.п.), специфичными для его проекта. Довольно важны и навыки SQL (11% вакансий). Математика, как мы знаем, тоже важна для DS, однако наличие математических знаний требуется лишь в 6% вакансий. Это объясняется тем, что на практике умение писать код, грамотно собирать и фильтровать данные и уже непосредственно обучать модели оказывается куда важнее каких-либо теоретических познаний. Новичку войти в отрасль не так-то просто - лишь в 3% вакансий работодатель готов принять специалиста без опыта работы. Основную часть вакансий по DS предоставляют такие крупные компании, как "Cбер", "ВТБ", "VK", "Яндекс" и др. Это тоже вполне ожидаемый результат: данные корпорации применяют ML в бизнесе и захватили достаточно большую часть рынка, их названия у всех на слуху.

# Общий вывод по проекту

Итак, мы внимательно изучили данные с сайта HeadHunter, провели детальный анализ вакансий и работников, более подробно остановившись на вакансиях из Data Science. Проведенные дополнительные исследования отмечены звездочками в каждом из разделов. В целом, все полученные результаты соответствуют ожиданиям и согласуются с ситуацией, наблюдаемой на реальном рынке труда. Однако все же можно предположить, что в процессе сбора данных были допущены некоторые ошибки/неточности, которые затруднили последующий анализ (например, неточный формат указания адреса регистрации работодателя).